# **Data Parsing and Preprocessing**

---

In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import dask.array as da
from hyss_util import *

**SET MPL DEFAULTS**

In [2]:
%matplotlib widget

In [3]:
# -- set mpl defaults
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gist_gray"
plt.rcParams["figure.figsize"] = (10, 5)

**LOAD THE HYPERSPECTRAL DATA**

To load the data, set the `LTCO_HSI0` and `LTCO_HSI1` environment variables,

In [4]:
# -- set the data paths
dpath0 = os.path.join(os.environ["LTCO_HSI0"], "full frame 20ms faster_VNIR.raw")
dpath1 = os.path.join(os.environ["LTCO_HSI1"], "night_00000.raw")

In [6]:
# -- calculate average brightness of HSI0
cube0 = read_hyper(dpath0)
print("average brightness is {0}".format(cube0.data.mean()))

reading and parsing full frame 20ms faster_VNIR.hdr...
reading full frame 20ms faster_VNIR.raw...
average brightness is 52.56469692011806


**DEFINE CLEANING FUNCTIONS**

In [ ]:
def sig_clipping_mean(arr, axis, niter=10, thr=3):
    
    # -- check for dask array
    if type(arr) is da.core.Array:
        return sig_clipping_mean_da(arr, axis, niter=niter, thr=thr)
    
    # -- convert to masked array
    arr = np.ma.masked_array(arr)
    
    # -- loop through iterations
    t00 = time.time()
    t0 = time.time()
    for ii in range(niter):
        print("cleaning axis {0} - |·".format(axis) + "·" * ii + " " * (niter - 1 - ii) + "| " 
              + "Elapsed time : {0:.2f}s | ".format(time.time() - t00) 
              + "Estimated remaining : {0:.2f}s".format((time.time() - t0) * (niter - ii + 1))
              + "\r", end="")
        
        t0 = time.time()
        avg = np.ma.mean(arr, axis=axis, keepdims=True)
        sig = np.ma.std(arr, axis=axis, keepdims=True)
        arr.mask = (arr > avg + thr * sig) | (arr < avg - thr * sig)
    print("")
    
    # -- subtract mean and return
    return (arr.data - np.ma.mean(arr, axis=axis, keepdims=True)).data

In [ ]:
def clean_hyper(arr, niter=10, thr=3, split=True):
    
    if split:
        srow = arr.shape[1] // 2
        print("cleaning top then bottom, split at row {0}".format(srow))
        
        return np.hstack(
            (sig_clipping_mean(sig_clipping_mean(arr[:, :srow, :], 2, niter=niter, thr=thr), 1, niter=niter, thr=thr), 
             sig_clipping_mean(sig_clipping_mean(arr[:, srow:, :], 2, niter=niter, thr=thr), 1, niter=niter, thr=thr))
                        )

    else:
        return sig_clipping_mean(sig_clipping_mean(arr, 2, niter=niter, thr=thr), 1, niter=niter, thr=thr)

---

## HSI0 Data Cleaning

In [ ]:
# -- read cleaned and registered if available, create if not
oname = "hsi0_clean_reg.npy"

if os.path.isfile(oname):
    # -- read in the HSI0 scan
    cube0 = read_hyper(dpath0)
    
    # -- read in the cleaned and "registered" cube
    clean0_tr = np.load(oname)

else:
    # -- read in the HSI0 scan
    cube0 = read_hyper(dpath0)

    # -- clean HSI0
    clean0 = clean_hyper(cube0.data, niter=2)
    
    # -- *roughly* align with HSI1
    clean0_tr = clean0[:, 238:933, :1087]
    
    # -- write to file
    np.save(oname, clean0_tr)

---

## Create masked aggregate spectrum

In [ ]:
# -- set up row/col grid
cc, rr = np.meshgrid(range(clean0_tr.shape[2]), range(clean0_tr.shape[1]))

# -- get pixels for patch
pchs = [[320, 320, 695, 475], 
        [165, 130, 400, 225]]

# -- make the patches
pval = -9999
for pch in pchs:
    pind = (rr >= pch[0]) & (rr < pch[2]) & (cc >= pch[1]) & (cc < pch[3]) 
    clean0_tr[:, pind] = pval

In [ ]:
# -- get aggregate spectrum
gind = clean0_tr[0] != pval
hsi0_spec_agg = clean0_tr[:, gind].mean(axis=1)
hsi0_spec_agg = (hsi0_spec_agg - hsi0_spec_agg.min()) / (hsi0_spec_agg.max() - hsi0_spec_agg.min())

---

## Correlation with LSPDD and NOAA templates

In [ ]:
# -- load templates
tmpl = np.load("final_spectra_20.npy")

# -- normalize templates
tmpl = (tmpl - tmpl.min(axis=0, keepdims=True)) / (tmpl.max(axis=0, keepdims=True) - tmpl.min(axis=0, keepdims=True))

# -- load wavelengths
tmpl_waves = np.load("lspdd_lab_spectra_wavelengths.npy")

In [ ]:
# -- plot the templates
plt.close("all")
fig, ax = plt.subplots(5, 4, sharex=True, sharey=True)
for ii in range(tmpl.shape[1]):
    ax[ii // 4, ii % 4].plot(tmpl_waves, tmpl[:, ii])
    ax[ii // 4, ii % 4].set_yticks(np.linspace(0, 1, 5))
    if ii // 4 == 4:
        ax[ii // 4, ii % 4].set_xlabel("wavelength [nm]")

In [ ]:
# -- set the minimum and maximum wavelengths
wmin = max(cube0.waves.min(), tmpl_waves.min())
wmax = min(cube0.waves.max(), tmpl_waves.max())

# -- create the interpolated wavelength array
waves_intp = cube0.waves[(cube0.waves >= wmin) & (cube0.waves <= wmax)]

# -- interpolate hsi0 spectrum and templates
hsi0_spec_agg_intp = np.interp(waves_intp, cube0.waves, hsi0_spec_agg)
tmpl_intp = np.array([np.interp(waves_intp, tmpl_waves, i) for i in tmpl.T]).T

In [ ]:
# -- plot the interpolated templates
plt.close("all")
fig, ax = plt.subplots(5, 4, sharex=True, sharey=True)
for ii in range(tmpl.shape[1]):
    ax[ii // 4, ii % 4].plot(tmpl_waves, tmpl[:, ii])
    ax[ii // 4, ii % 4].plot(waves_intp, tmpl_intp[:, ii], ".", ms=1)
    ax[ii // 4, ii % 4].set_yticks(np.linspace(0, 1, 5))
    if ii // 4 == 4:
        ax[ii // 4, ii % 4].set_xlabel("wavelength [nm]")

In [ ]:
# -- plot the HSI0 aggregate interpolated spectrum
plt.close("all")
plt.plot(cube0.waves, hsi0_spec_agg)
plt.plot(waves_intp, hsi0_spec_agg_intp, ".", ms=2)
plt.xlabel("wavelength [nm]")
plt.show()

In [ ]:
np.linalg.solve?

In [ ]:
# -- correlate by solving the normal equation
A = np.hstack([tmpl_intp, np.ones((tmpl_intp.shape[0], 1))]).T
coef = np.linalg.inv(A @ A.T) @ (hsi0_spec_agg_intp @ A.T)

In [ ]:
# -- plot the model
plt.close("all")
plt.plot(waves_intp, hsi0_spec_agg_intp, lw=2)
plt.plot(waves_intp, coef @ A, lw=2)
plt.xlabel("wavelength [nm]")
plt.show()

In [ ]:
np.interp?

In [ ]:

np.interp()

In [ ]:
tmpl.shape

In [ ]:
plt.close("all")
plt.plot(cube0.waves, hsi0_spec_agg)

In [ ]:
clean0_tr.shape

In [ ]:
cc.shape

In [ ]:
cc

In [ ]:
imgL = clean0_tr.mean(axis=0)

In [ ]:
plt.close("all")
plt.imshow(imgL, aspect="auto", clim=[-2, 3])

In [ ]:
# -- write 8-bit
write_hyper_8bit("hsi0_clean_8bit_clip_0050.npy", clean0, 50)

In [ ]:
# -- load the 8-bit data
clean0_8b = np.load("hsi0_clean_8bit_clip_0050.npy")
imgL0_8b = clean0_8b.mean(axis=0)
spec0_8b = clean0_8b.mean(axis=(1, 2))

In [ ]:
# -- plot the luminosity image and integrated spectrum
plt.close("all")
fig = plt.figure(figsize=(9, 6))
bax = fig.add_axes((0.15, 0.65, 0.8, 0.25))
# bax.plot(cube0.waves, spec0_8b)
bax.plot(spec0_8b)
bax.set_xlabel("wavelength [nm]")
bax.set_ylabel("intensity [arb units]")
tax = fig.add_axes((0.15, 0.1, 0.8, 0.45))
tax.imshow(imgL0_8b, aspect="auto", clim=(150, 175))

---

## HSI1 Data Cleaning

In [ ]:
# -- read in the HSI1 scan
cube1 = read_hyper(dpath1)

In [ ]:
# -- clean HSI1
clean1 = clean_hyper(cube1.data[:, ::5, ::5], niter=3)

In [ ]:
# -- figure out max clipping that leaves 0.1% of values unaffected
((clean1 - clean1.min()) > 205).sum() / clean1.size

In [ ]:
# -- write 8-bit
write_hyper_8bit("hsi1_clean_8bit_clip_0205.npy", clean1, 205)

In [ ]:
# -- load the 8-bit data
clean1_8b = np.load("hsi1_clean_8bit_clip_0205.npy")
imgL1_8b = clean1_8b.mean(axis=0)
spec1_8b = clean1_8b.mean(axis=(1, 2))

In [ ]:
imgL1_8b.min()

In [ ]:
# -- plot the luminosity image and integrated spectrum
plt.close("all")
fig = plt.figure(figsize=(9, 6))
bax = fig.add_axes((0.15, 0.65, 0.8, 0.25))
bax.grid(True)
# bax.plot(cube1.waves, spec1_8b)
bax.plot(spec1_8b)
bax.set_xlabel("wavelength [nm]")
bax.set_ylabel("intensity [arb units]")
tax = fig.add_axes((0.15, 0.1, 0.8, 0.45))
tax.imshow(imgL1_8b, aspect="auto", clim=(160, 180))

In [ ]:
plt.close("all")
plt.plot(cube1.waves, clean1.mean(axis=(1, 2)))

In [ ]:
arr = cube0.data.astype(float)

In [ ]:
darr = da.array(cube0.data.astype(float))

In [ ]:
t0 = time.time()
img0 = arr.mean(axis=0)
time.time() - t0

In [ ]:
t0 = time.time()
img0 = darr.mean(axis=0).compute()
time.time() - t0

In [ ]:
img0[100, 201].compute()

In [ ]:
arr = da.array(cube0.data.astype(float))

In [ ]:
foo = sig_clipping_mean(arr, 2)

In [ ]:
foo

In [ ]:
arr

In [ ]:
img0 = arr.mean(axis=0).compute()

In [ ]:
img1 = foo.mean(axis=0).compute()

In [ ]:
plt.close("all")
plt.imshow(img0, aspect="auto", clim=[0, 100])

In [ ]:
plt.close("all")
plt.imshow(img1, aspect="auto", clim=[-10, 10])

In [ ]:
foo

In [ ]:
imgL0 = 

In [ ]:
t0 = time.time()
avg = da.ma.average(arr, axis=2)
time.time() - t0

In [ ]:
da.expand_dims(avg, 2).shape

In [ ]:
# -- perform 10-fold 3-sigma clipping across columns
t0 = time.time()
clean_col = sig_clipping_col(cube0.data)
print("time to clean across columns : {0:.2f}".format(time.time() - t0))